In [1]:
import gensim
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import words
from nltk.corpus import wordnet
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


- After loading the data, we clean it

In [2]:
data=pd.read_csv("Emails.csv")
data.fillna(' ',inplace=True)

In [3]:
raw=data['ExtractedSubject'] + ' ' + data['ExtractedBodyText']

### CLEANING

In [4]:
def removeStopWords(String):
    stopwordlist = set(stopwords.words("english"))
    toRemove = ['fm','am',"n't", 'sent', 'fw', 'pm', 'fwd', 're','cc', 'attachments', "rv", "al", "aq", "la", "ben"]
    stopwordlist.update(toRemove)  
    liste=[word for word in String if word.lower() not in stopwordlist]
    return liste

def removeChar(string):
    return re.sub("[^a-zA-Z]+", " ", string)

def lowerCase(string):
    return string.lower()

In [5]:
i=0
for mail in raw:    
    cleaned_raw=mail
    cleaned_raw=lowerCase(cleaned_raw)
    cleaned_raw=nltk.word_tokenize(cleaned_raw)
    cleaned_raw=removeStopWords(cleaned_raw)
    cleaned_raw=' '.join(cleaned_raw)
    cleaned_raw=removeChar(cleaned_raw)
    cleaned_raw=[token for token in cleaned_raw.split() if len(token)>1]
    raw[i]=cleaned_raw
    i+=1

In [7]:
raw[1006]

['hrod',
 'clintonemail',
 'com',
 'thursday',
 'october',
 'pverveer',
 'bravo',
 'many',
 'miles',
 'go',
 'thx',
 'much',
 'back',
 'next',
 'week']

- We define a dictionary which associate an index to every word
- We also define the corpus in which we will do our analysis

In [8]:
dictionary = gensim.corpora.Dictionary(raw)

In [9]:
corpus = [dictionary.doc2bow(token) for token in raw]

- Using the gensim library, we define the latant dirichlet model

In [10]:
topic=[5,10,20,30,40,50]
for i in topic:
    model = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)
    print("------------------------------------",i, "topics "'------------------------------------')
    print("---------------------------------------------------------------------------------------")
    print(model.show_topics(num_topics=i))

------------------------------------ 5 topics ------------------------------------
---------------------------------------------------------------------------------------
[(0, '0.007*"obama" + 0.006*"would" + 0.005*"said" + 0.005*"one" + 0.004*"new" + 0.004*"israel" + 0.004*"party" + 0.003*"us" + 0.003*"government" + 0.003*"american"'), (1, '0.007*"call" + 0.005*"ap" + 0.004*"see" + 0.004*"vote" + 0.004*"also" + 0.004*"part" + 0.003*"pls" + 0.003*"get" + 0.003*"would" + 0.003*"office"'), (2, '0.008*"fyi" + 0.006*"state" + 0.004*"think" + 0.004*"work" + 0.004*"report" + 0.003*"good" + 0.003*"see" + 0.003*"president" + 0.003*"new" + 0.003*"us"'), (3, '0.020*"state" + 0.019*"call" + 0.014*"gov" + 0.007*"talk" + 0.006*"ok" + 0.006*"tomorrow" + 0.005*"secretary" + 0.005*"com" + 0.005*"today" + 0.005*"clintonemail"'), (4, '0.010*"secretary" + 0.009*"office" + 0.006*"state" + 0.005*"meeting" + 0.005*"time" + 0.005*"president" + 0.004*"room" + 0.004*"department" + 0.004*"new" + 0.003*"private"

- Looking at the results, we settle for 30 as number of topics since it seems meaningful
- Indeed, we can see that it separates topics
- for instance ['israeli', 'palestinian', 'israel', 'branch', 'arab', 'abbas', 'funded', 'state', 'ap', 'lewis', 'film', 'hamas', 'one', 'ex', 'honduras', 'soldier', 'january', 'made', 'well', 'peace'] for the israeli-palestinian conflict

In [14]:
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word=dictionary)

In [15]:
for x in range(model.num_topics):
    line = [l for l, _ in model.show_topic(x, topn=20)]
    print(x+1,':',line)

1 : ['tea', 'favorable', 'hrod', 'call', 'com', 'ashton', 'clintonemail', 'state', 'gov', 'print', 'today', 'may', 'monday', 'pls', 'pis', 'saturday', 'madame', 'conspiracy', 'cheryl', 'declassify']
2 : ['secretary', 'office', 'state', 'meeting', 'room', 'department', 'en', 'route', 'arrive', 'private', 'depart', 'conference', 'residence', 'call', 'time', 'daily', 'staff', 'airport', 'assistant', 'floor']
3 : ['boehner', 'house', 'speaker', 'arizona', 'state', 'office', 'federal', 'pelosi', 'burma', 'boy', 'bill', 'san', 'floor', 'ventures', 'nancy', 'suu', 'aung', 'season', 'scott', 'koch']
4 : ['said', 'obama', 'would', 'israel', 'american', 'new', 'government', 'one', 'people', 'security', 'political', 'two', 'world', 'united', 'states', 'peace', 'could', 'first', 'also', 'women']
5 : ['obama', 'said', 'mr', 'white', 'would', 'mcchrystal', 'time', 'president', 'one', 'party', 'new', 'think', 'house', 'state', 'also', 'women', 'public', 'national', 'us', 'secretary']
6 : ['israeli', 